# Building a Deployment Environment
Create an S3 bucket for the build, deploy the Lambda binaries and CloudFormation template, and run CloudFormation.

# [Required] 
For running Cloud Formation, some roles below are needed:  
- AmazonS3FullAccess or ListBucket/GetObject/DeleteObject/PutObject  
- AWSLambdaFullAccess  
- IAMFullAccess  
- AWSCloudTrailFullAccess  
- CloudWatchLogsFullAccess  
- AWSStepFunctionsFullAccess  
- AmazonSageMakerFullAccess  
- AWSCloudFormationFullAccess  

## Create S3 bucket  
Create an S3 bucket to store the Lambda binary file and the Cloud Formation template file to be used for environment preparation.

In [6]:
import boto3

In [7]:
session = boto3.Session()
region = session.region_name
account_id = session.client('sts').get_caller_identity().get('Account')
bucket_name = f'workshop-timeseries-retail-{account_id}-deploy'
s3 = boto3.client('s3')

In [8]:
bucket_name

'workshop-timeseries-retail-673990568411-deploy'

In [ ]:
region

In [9]:
s3.create_bucket(Bucket=bucket_name,
                CreateBucketConfiguration={'LocationConstraint':region}
                )

{'ResponseMetadata': {'RequestId': '0SRC9CSTMMPXCP51',
  'HostId': 'TVQ10O5qJZI8OlB/28vESb3t9p5Ux3eRO5WWJn+4eIqLwSk3Ug5slpoQ4c4C1frAwr7c2OljNyk=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'TVQ10O5qJZI8OlB/28vESb3t9p5Ux3eRO5WWJn+4eIqLwSk3Ug5slpoQ4c4C1frAwr7c2OljNyk=',
   'x-amz-request-id': '0SRC9CSTMMPXCP51',
   'date': 'Thu, 18 Feb 2021 23:27:53 GMT',
   'location': 'http://workshop-timeseries-retail-673990568411-deploy.s3.amazonaws.com/',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Location': 'http://workshop-timeseries-retail-673990568411-deploy.s3.amazonaws.com/'}

## Put Lambda zip files to S3

In [10]:
import glob

files = glob.glob("lambdas_deploy/*")
for file in files:
    print(file)
    s3.upload_file(file, bucket_name, file)

lambdas_deploy\createdataset.zip
lambdas_deploy\createdatasetgroup.zip
lambdas_deploy\createdatasetimport.zip
lambdas_deploy\createforecast.zip
lambdas_deploy\createforecastexportjob.zip
lambdas_deploy\createpredictor.zip
lambdas_deploy\getstatusforecast.zip
lambdas_deploy\getstatusforecastexportjob.zip
lambdas_deploy\getstatusimport.zip
lambdas_deploy\getstatuspredictor.zip


## Run CloudFormation

In [11]:
s3.upload_file('cfn_template.yaml', bucket_name, 'cfn_template.yaml')

In [12]:
cf = boto3.client('cloudformation')
res = cf.create_stack(
    StackName=('workshop-timeseries-retail'),
    TemplateURL=f'https://workshop-timeseries-retail-{account_id}-deploy.s3.amazonaws.com/cfn_template.yaml',
    Parameters=[
        {
            'ParameterKey': 'BucketName',
            'ParameterValue': bucket_name
        },
        {
            'ParameterKey': 'Account',
            'ParameterValue': account_id
        },
    ],
    Capabilities=[
        'CAPABILITY_IAM',
        'CAPABILITY_AUTO_EXPAND',
    ],
)

## Next
Wait for CloudFormation to complete. When it is complete, switch to the sagemaker notebook instance named workshop-timeseries-retail-ForecastNotebook created by the Cloud Formation and run 1_build_statemachine.ipynb.